In [4]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw, dtw_path
from sklearn.metrics import accuracy_score
from scipy.spatial import distance as dist
from scipy.stats import norm

from reader import get_x_label, get_class_x
from average import amplitude_avg, dtw_avg, soft_dtw_avg

In [14]:
file_name = 'Coffee'

In [15]:
class_x = get_class_x(file_name + '_TRAIN')
class_avg = dtw_avg(class_x)

In [16]:
dtw_stat = [[list() for i in range(len(class_x[0][0]))] for _ in range(len(class_x))]
for i in range(class_x.shape[0]):
    for j in range(len(class_x[i])):
        path, dist = dtw_path(class_avg[i], class_x[i][j])
        for avg_pos, x_pos in path:
            dtw_stat[i][avg_pos].append(class_x[i][j][x_pos])
dtw_stat = np.array(dtw_stat)
for i in range(dtw_stat.shape[0]):
    for j in range(dtw_stat.shape[1]):
        dtw_stat[i] = np.array(dtw_stat[i])

In [17]:
dtw_mean = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
dtw_sd = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
for class_label in range(dtw_stat.shape[0]):
    for point in range(dtw_stat.shape[1]):
        stat = np.array(dtw_stat[class_label][point])
        dtw_mean[class_label][point].append(stat.mean())
        dtw_sd[class_label][point].append(stat.std())

In [18]:
test_x, test_label = get_x_label(file_name + '_TEST')

In [19]:
bay_predict = []
dist_predict = []
count = 0
for x in test_x:
    min_log_prob = 0
    min_dist = 100000
    bay_nearest_class = -1
    dist_nearest_class = -1
    for j in range(len(class_avg)):
        # Class j
        path, dist = dtw_path(x, class_avg[j])
        log_prob = 1
        probs = [list() for _ in range(len(x))]
        for x_pos, avg_pos in path:
            # prob = norm(dtw_mean[j], dtw_sd[j]).pdf(class_avg[j][avg_pos][0])
            # prob = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).pdf(x[x_pos])
            prob = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).pdf(x[x_pos])
            if prob < 10**(-310):
                prob = 10**(-310)
            probs[x_pos].append(prob)
        for pos_probs in probs:
#             print(len(pos_probs))
            max_pos_prob = min(pos_probs)
            #print(max_pos_prob)
            log_prob *= max_pos_prob
#         print(log_prob)
        if log_prob > min_log_prob:
            min_log_prob = log_prob
            bay_nearest_class = j
        if dist < min_dist:
            min_dist = dist
            dist_nearest_class = j
    bay_predict.append(bay_nearest_class)
    dist_predict.append(dist_nearest_class)
    print(count, ':', bay_nearest_class, dist_nearest_class, test_label[count]-1)
    count += 1

0 : -1 1 -1
1 : 1 1 -1
2 : 1 1 -1
3 : 1 1 -1
4 : 1 1 -1
5 : 1 1 -1
6 : 1 1 -1
7 : 1 1 -1
8 : 0 1 -1
9 : 0 0 -1
10 : -1 0 -1
11 : 0 0 -1
12 : 1 1 -1
13 : 1 0 -1
14 : 1 1 -1
15 : 0 0 0
16 : 1 1 0
17 : 0 0 0
18 : 1 1 0
19 : 0 0 0
20 : 0 0 0
21 : 1 1 0
22 : 1 0 0
23 : 0 0 0
24 : 1 0 0
25 : 0 0 0
26 : 0 0 0
27 : 0 0 0


In [20]:
accuracy_score(test_label, bay_predict)

0.2857142857142857